In [1]:
import torch
import torch.nn as nn
import os, time
import numpy as np
from model.net import FashionNet
from load_dataset import custom_dset, collate_fn, denorm
from torch.utils.data import DataLoader
import cv2
import argparse
from utils import AverageMeter, AverageAcc
from tqdm import tqdm
from focal_loss import *


In [2]:
from IPython.display import display, HTML
import pandas as pd

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [3]:
def test(net, test_loader, accuracy):


    losses = AverageMeter()

    criterion = FocalLoss(gamma=2, alpha=0.25)
    label_map = { v:k for k,v in test_loader.dataset.label_map.items()} 

    bar = tqdm(total=len(test_loader))

    with torch.no_grad():
        for it, (imgs, labels, img_ids) in enumerate(test_loader):

            imgs = imgs.cuda()
            labels = labels.cuda()

            # run input through the network
            preds = net(imgs)
            loss = criterion(preds, labels)


            # measure accuracy and record loss
            accuracy.update(preds.data, labels.data, topk=(1, 5))
            losses.update(loss.item(), imgs.size(0))
            bar.update()
            if it % 100 == 0:
                # save to disk first batch of results for every epoch
                for i,im in enumerate(imgs):
                    im = im.data.cpu().numpy()
                    im = denorm(im.transpose(1,2,0))[...,::-1]

                    _, inds = torch.max(preds, 1)
                    pred = label_map[inds[i].item()]
                    gt_label = label_map[labels[i].item()]

                    fname =  str(img_ids[i]) + '__' 'Gt:' + gt_label + 'Pred:' + pred + '.jpg'
                    cv2.imwrite(os.path.join(out_path, fname),  im)

        bar.close()



In [4]:
out_path = './data/show_test'
if not os.path.isdir(out_path):
    os.mkdir(out_path)


# check checkpoints folder for model_name
model_name = 'pnet_4.pth'

# get the current working directory
root = os.getcwd()

# load model
model_path = os.path.join(root, 'checkpoints', model_name)
model = torch.load(model_path)
label_map = model['label_map']
net = FashionNet(len(label_map))
net.load_state_dict(model['net'])

net = net.cuda()
net.eval()


test_pretrain = custom_dset('test_pretrain', label_map)
print('Pretrain Test Dataset of size: {}'.format(len(test_pretrain)))
test_loader = DataLoader(test_pretrain, batch_size=24, shuffle=False, collate_fn=collate_fn, num_workers=4)

accuracy = AverageAcc(label_map)
test(net, test_loader, accuracy)

# get accuracies per class and the averages 
topk_dict, top1, top5 = accuracy()



  0%|          | 0/520 [00:00<?, ?it/s]

Pretrain Test Dataset of size: 12476


100%|██████████| 520/520 [01:45<00:00,  6.89it/s]


In [5]:

# collect accuracies into lists for printing purpose 
lc = [k for k in label_map]
t1 = [v[1] for k,v in topk_dict.items()]
t2 = [v[5] for k,v in topk_dict.items()]
avg = [round(top1, 2), round(top5, 2)]


In [6]:
trainset =  custom_dset('train_pretrain')
lc_train = [trainset.freq[c] for c in lc]
# display train set statitstics 
df_dset = pd.DataFrame(np.vstack((lc, lc_train)).transpose(), index=['Trainset']*19,
                       columns=['Category', 'Data examples'])
display(df_dset)


# display dataset statitstics 
df_dset = pd.DataFrame(np.vstack( (lc, [v[1] for _,v in accuracy.count.items()])).transpose(), columns=['Category', 'Data examples'],
                       index=['Testset']*19)
display(df_dset)

# concatenate accuracy lists in numpy array of shape (num_classes, num_columns)
df_class = pd.DataFrame(np.vstack( (lc, t1, t2)).transpose(), columns=['Category', 'Top1', 'Top5'], index=['TestAcc']*19)
df_avg = pd.DataFrame(np.array(avg).reshape(1,2), columns=['Average Top1', 'Average Top5'])

# display the accuracies 
display(df_class)
display(df_avg)


,Category,Data examples
Trainset,Backpacks,212
Trainset,Belts,431
Trainset,Briefs,568
Trainset,Casual Shoes,1354
Trainset,Flip Flops,451
Trainset,Formal Shoes,318
Trainset,Handbags,851
Trainset,Heels,670
Trainset,Jeans,288
Trainset,Kurtas,986


,Category,Data examples
Testset,Backpacks,417
Testset,Belts,269
Testset,Briefs,156
Testset,Casual Shoes,1113
Testset,Flip Flops,331
Testset,Formal Shoes,242
Testset,Handbags,635
Testset,Heels,467
Testset,Jeans,232
Testset,Kurtas,618


,Category,Top1,Top5
TestAcc,Backpacks,0.85,1.0
TestAcc,Belts,0.97,0.99
TestAcc,Briefs,0.9,1.0
TestAcc,Casual Shoes,0.77,1.0
TestAcc,Flip Flops,0.69,0.99
TestAcc,Formal Shoes,0.7,0.98
TestAcc,Handbags,0.95,1.0
TestAcc,Heels,0.94,1.0
TestAcc,Jeans,0.93,0.97
TestAcc,Kurtas,0.91,1.0


,Average Top1,Average Top5
0,0.87,0.99


In [7]:
top1


0.8726315789473683